In [1]:
if 'google.colab' in str(get_ipython()):
    %matplotlib inline
else:
    %matplotlib notebook

In [2]:
import time
from funtion_file import *

In [3]:
matrix_params = {
    'mat_size': (5,5),
    'strength_kernel': (5, 3), 
    'min_max_line_size': [(1,3),(1,3)],
    'rotate': True,
    'fades_per_mat': 8,
    'new_background': False,
    'shape': 'line',  # 'line', 'triangle', 'face'
}

matrix_lister = MatrixLister(**matrix_params)

In [4]:
model, callbacks = matrix_lister.init_model(32, 128, 'none')  # auto, line, triangle, none

In [5]:
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime

log_dir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
batch_size = 500
batch_num = 10
epochs = 100

generator = matrix_lister.init_generator(batch_size, batch_num)


start = time.time()
hist = model.fit(generator, epochs=epochs, callbacks=tensorboard_callback)
print(time.time() - start)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 348ms/step - loss: 0.0730 - precision: 0.7020 - recall: 0.8762
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 345ms/step - loss: 0.0714 - precision: 0.7140 - recall: 0.8771
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 346ms/step - loss: 0.0695 - precision: 0.7123 - recall: 0.8841
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 346ms/step - loss: 0.0682 - precision: 0.7155 - recall: 0.8859
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 347ms/step - loss: 0.0680 - precision: 0.7209 - recall: 0.8850
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 356ms/step - loss: 0.0681 - precision: 0.7191 - recall: 0.8862
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 360ms/step - loss: 0.0670 - precision: 0.7259 - recall: 0.8867
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 356ms/step - loss: 0.0670 - precision: 0.7209 - recall: 0.8874
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 357ms/step - loss: 0.0664 - precision: 0.7226 - recall: 0.8881
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 356ms/step -

In [10]:
matrix_lister.save_model(model, 'auto')  # auto, line, triangle, none

Saved line weights


In [15]:
%matplotlib notebook
matrix_lister.display_frames(model, 16)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


<IPython.core.display.Javascript object>

In [ ]:
ani = matrix_lister.plot_matrices(model, 50, interval=500)
plt.show()

In [ ]:
plot_training_history(hist, with_val=False)